# Text-to-SQL Translation with Various Neural Networks

## Introduction

Generating SQL queries from natural language has long been a popular and useful task, attracting considerable interest. Most of the existing models are based on Long Short-Term Memory (LSTM) networks with various attention mechanisms. Recently, a new neural network architecture called Transformer has been proven to gain higher accuracy in neural machine translation scenarios. In addition, CNN has been proven to be efficient for text classification.

In this project, we aim to explore different existing text-to-SQL models and analyze them, as well as implement our model using Transformer and CNN. We evaluate how accuracy would be affected. Our experiments find that introducing syntactical structures of SQL to neural networks can be helpful for text-to-SQL translation. Furthermore, for aggregator prediction, transformer and CNN encoders for questions could achieve comparable results to LSTM encoders.


## Related Work

Synthesizing SQL queries has long been a popular research topic. Seq2SQL has been the first deep neural network-based approach to solve this problem. They constructed WikiSQL, one of the largest natural language query to SQL datasets, and developed a deep neural network for translation.

While most of the existing models are based on LSTM networks, recently new models have been proposed and proved efficient in some specific tasks. Transformer, a simple network only based on attention mechanisms, achieves state-of-the-art performance in English-to-German and English-to-French translation tasks. CNN has also been proven to show excellence in text classification tasks.

In this work, we introduce these models on the SQL translation problem, exploring how they perform in this scenario. With limited time and resources, we only focus on the 'SELECT' clause in SQL grammar.


## Approach

We explore various models for the text-to-SQL task, including a baseline LSTM-based model, Seq2SQL, SQLNet, and modified versions incorporating BERT embeddings, Transformer encoders, and CNN encoders.


### Baseline Model

Text-to-SQL can be viewed as a language translation problem, so we implemented an LSTM-based neural machine translation model as our baseline. We implemented a bidirectional LSTM encoder and a unidirectional LSTM decoder. The model computes multiplicative attention using encoder hidden states and concatenates attention output with decoder hidden states. Pretrained GloVe word embeddings are used and set fixed during the training process.

In [1]:
!pip -q install torch transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 20.6 MB/s eta 0:00:00


## Step 1: Load and Preprocess the Dataset

In [2]:
from datasets import load_dataset
from transformers import BertTokenizer, BertModel, AdamW
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd

In [3]:
# Load the dataset
dataset = load_dataset("wikisql")

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
def preprocess_function(batch):
    input_ids = []
    attention_masks = []
    labels = []

    for example in batch['question']:
        # Tokenize the questions
        inputs = tokenizer(example, padding="max_length", truncation=True)

        input_ids.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])

    for example in batch['sql']:
        # Extract labels for SELECT columns
        labels.append(example['sel'])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': labels
    }

# Apply preprocessing
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Convert to PyTorch tensors
def format_to_torch(batch):
    return {
        'input_ids': torch.tensor(batch['input_ids']),
        'attention_mask': torch.tensor(batch['attention_mask']),
        'labels': torch.tensor(batch['labels'])
    }

encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Create DataLoader
train_loader = DataLoader(encoded_dataset['train'], batch_size=16, shuffle=True)
valid_loader = DataLoader(encoded_dataset['validation'], batch_size=16)
test_loader = DataLoader(encoded_dataset['test'], batch_size=16)

Map:   0%|          | 0/15878 [00:00<?, ? examples/s]

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [5]:
class TransformerSQLModel(nn.Module):
    def __init__(self, transformer_model_name='bert-base-uncased', num_classes=5):
        super(TransformerSQLModel, self).__init__()
        self.transformer = BertModel.from_pretrained(transformer_model_name)
        self.classifier = nn.Linear(self.transformer.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        transformer_outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = transformer_outputs.pooler_output
        logits = self.classifier(pooled_output)
        return logits

# Initialize model
model = TransformerSQLModel()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [10]:
!export CUDA_LAUNCH_BLOCKING=1

In [16]:
num_epochs = 3
best_accuracy = 0

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = eval_model(model, valid_loader, criterion, device)

    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}')
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model_state.bin')

1

In [ ]:
model.load_state_dict(torch.load('best_model_state.bin'))
test_loss, test_accuracy = eval_model(model, test_loader, criterion, device)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [ ]:
class SQLInference:
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device

    def infer_sql(self, query, table_schema):
        self.model.eval()

        # Tokenize the input query
        inputs = self.tokenizer(query, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = inputs['input_ids'].to(self.device)
        attention_mask = inputs['attention_mask'].to(self.device)

        # Perform inference
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        # Get the predicted SELECT column
        _, pred_select_col = torch.max(outputs, dim=1)
        pred_select_col = pred_select_col.item()

        # Construct the SQL query
        select_col_name = table_schema['header'][pred_select_col]
        sql_query = f"SELECT {select_col_name} FROM table"

        return sql_query

# Initialize the inference object
sql_inference = SQLInference(model, tokenizer, device)

# Example table schema and natural language query
table_schema = {
    "header": ["State/territory", "Text/background colour", "Format", "Current slogan", "Current series", "Notes"],
    "rows": [ ... ]  # You can include the actual rows if needed for more complex SQL generation
}
query = "Tell me what the notes are for South Australia"

# Generate SQL query
sql_query = sql_inference.infer_sql(query, table_schema)

print("Generated SQL Query:", sql_query)
